In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.utils import set_seed, jprint

set_seed(89)

In [3]:
pd.options.display.float_format = '{:,.3f}'.format

In [4]:
from tqdm.auto import tqdm
tqdm.pandas()

In [5]:
def load_result_dataframe(pattern: str = "our-method-report-20240821-*.jsonl") -> pd.DataFrame:
    dfs = [pd.read_json(filename, orient='records', lines=True) for filename in Path(".").glob(pattern)]

    for run, df in enumerate(dfs):
        df["run"] = run + 1

    return pd.concat(dfs, ignore_index=True) 

In [6]:
raw_df = load_result_dataframe()
raw_df

,qdecomp,context,retrieval,top_k,qa,run,exact_match,f1
0,False,paragraphs,Sparse,3,Standard,1,0.180,0.274
1,False,paragraphs,Sparse,5,Standard,1,0.160,0.266
2,False,paragraphs,Sparse,10,Standard,1,0.080,0.138
3,False,paragraphs,Dense,3,Standard,1,0.240,0.319
4,False,paragraphs,Dense,5,Standard,1,0.180,0.257
...,...,...,...,...,...,...,...,...
67,True,paragraphs,Sparse,5,CTE,3,0.490,0.610
68,True,paragraphs,Sparse,10,CTE,3,0.510,0.600
69,True,paragraphs,Dense,3,CTE,3,0.560,0.661
70,True,paragraphs,Dense,5,CTE,3,0.520,0.620


In [7]:
raw_df.rename(
    columns={
        'qdecomp': 'Q-decomp',
        'context': 'Context',
        'retrieval': 'Retrieval',
        'top_k': 'Top-k',
        'qa': 'Prompting',
        'run': 'Run',
        'exact_match': "EM",
        'f1': "F1",
    }, 
    inplace=True,
)

In [8]:
df = raw_df.drop(columns=["Context"])

In [9]:
agg_df = df.drop(columns=['Run']).groupby(['Q-decomp', 'Prompting', 'Retrieval', 'Top-k']).mean()
agg_df

EM    F1
Q-decomp Prompting Retrieval Top-k            
False    CTE       Dense     3     0.260 0.369
                             5     0.310 0.408
                             10    0.347 0.477
                   Sparse    3     0.170 0.278
                             5     0.277 0.384
                             10    0.347 0.473
         Standard  Dense     3     0.233 0.316
                             5     0.180 0.259
                             10    0.137 0.194
                   Sparse    3     0.190 0.283
                             5     0.157 0.259
                             10    0.123 0.185
True     CTE       Dense     3     0.547 0.654
                             5     0.503 0.605
                             10    0.537 0.644
                   Sparse    3     0.533 0.627
                             5     0.493 0.608
                             10    0.500 0.593
         Standard  Dense     3     0.483 0.572
                             5     0.420 0.494
                             10    0.370 0.448
                   Sparse    3     0.433 0.500
                             5     0.383 0.448
                             10    0.360 0.423

In [10]:
with open("experiment-results-detailed.tex", 'w') as f:
    f.write(df.sort_values(['Q-decomp', 'Prompting', 'Retrieval', 'Top-k', 'Run']).to_latex(index=False, float_format='%.3f'))

In [11]:
with open("experiment-results-agg.tex", 'w') as f:
    f.write(agg_df.sort_values(['Q-decomp', 'Prompting', 'Retrieval', 'Top-k']).to_latex(float_format='%.3f'))

## Explore

In [12]:
df = agg_df.reset_index()
df.head()

,Q-decomp,Prompting,Retrieval,Top-k,EM,F1
0,False,CTE,Dense,3,0.260,0.369
1,False,CTE,Dense,5,0.310,0.408
2,False,CTE,Dense,10,0.347,0.477
3,False,CTE,Sparse,3,0.170,0.278
4,False,CTE,Sparse,5,0.277,0.384


### Inspect

In [13]:
qd_mask = df['Q-decomp']
context_mask = df['Context'] == 'paragraphs'
retrieval_mask = df['Retrieval'].isin(['Dense', 'Sparse'])
df.loc[context_mask & retrieval_mask ].sort_values('F1', ascending=False)

KeyError: 'Context'

In [ ]:
df.loc[context_mask & retrieval_mask ].sort_values('F1', ascending=False).set_index(['Q-decomp', 'Context', 'QA Prompting', 'Retrieval', 'top-k'])

EM    F1
Q-decomp Context  QA Prompting Retrieval top-k            
True     triplets Standard     Dense     3     0.433 0.509
                                         10    0.413 0.503
                                         5     0.420 0.499
False    triplets Standard     Dense     5     0.223 0.351
                                         3     0.220 0.346
                                         10    0.187 0.308

In [ ]:
qd_mask = df['Q-decomp']
context_mask = df['Context'] == 'paragraphs'
retrieval_mask = df['Retrieval'].isin(['Perfect'])
df.loc[context_mask & retrieval_mask ].sort_values('F1', ascending=False)

,Q-decomp,Context,QA Prompting,Retrieval,top-k,EM,F1
4,False,paragraphs,CTE,Perfect,2,0.627,0.736
36,True,paragraphs,CTE,Perfect,2,0.587,0.696
44,True,paragraphs,Standard,Perfect,2,0.513,0.620
12,False,paragraphs,Standard,Perfect,2,0.507,0.616


In [ ]:
agg_df.loc[idx[:, 'paragraphs', :, :]]

EM    F1
Q-decomp QA Prompting Retrieval top-k            
False    CTE          Dense     3     0.307 0.404
                                5     0.340 0.438
                                10    0.337 0.476
                      Dummy     20    0.517 0.599
                      Perfect   2     0.627 0.736
                      Sparse    3     0.173 0.279
                                5     0.283 0.385
                                10    0.337 0.432
         Standard     Dense     3     0.243 0.314
                                5     0.210 0.292
                                10    0.153 0.204
                      Dummy     20    0.127 0.167
                      Perfect   2     0.507 0.616
                      Sparse    3     0.203 0.295
                                5     0.163 0.253
                                10    0.173 0.243
True     CTE          Dense     3     0.523 0.624
                                5     0.503 0.610
                                10    0.500 0.612
                      Dummy     20    0.523 0.615
                      Perfect   2     0.587 0.696
                      Sparse    3     0.510 0.615
                                5     0.517 0.619
                                10    0.493 0.585
         Standard     Dense     3     0.470 0.565
                                5     0.407 0.481
                                10    0.380 0.443
                      Dummy     20    0.177 0.221
                      Perfect   2     0.513 0.620
                      Sparse    3     0.433 0.498
                                5     0.377 0.446
                                10    0.357 0.411

In [ ]:
idx = pd.IndexSlice
print(agg_df.loc[idx[:, 'paragraphs', :, :]].to_latex(float_format='%.3f'))

\begin{tabular}{llllrr}
\toprule
 &  &  &  & EM & F1 \\
Q-decomp & QA Prompting & Retrieval & top-k &  &  \\
\midrule
\multirow[t]{16}{*}{False} & \multirow[t]{8}{*}{CTE} & \multirow[t]{3}{*}{Dense} & 3 & 0.307 & 0.404 \\
 &  &  & 5 & 0.340 & 0.438 \\
 &  &  & 10 & 0.337 & 0.476 \\
\cline{3-6}
 &  & Dummy & 20 & 0.517 & 0.599 \\
\cline{3-6}
 &  & Perfect & 2 & 0.627 & 0.736 \\
\cline{3-6}
 &  & \multirow[t]{3}{*}{Sparse} & 3 & 0.173 & 0.279 \\
 &  &  & 5 & 0.283 & 0.385 \\
 &  &  & 10 & 0.337 & 0.432 \\
\cline{2-6} \cline{3-6}
 & \multirow[t]{8}{*}{Standard} & \multirow[t]{3}{*}{Dense} & 3 & 0.243 & 0.314 \\
 &  &  & 5 & 0.210 & 0.292 \\
 &  &  & 10 & 0.153 & 0.204 \\
\cline{3-6}
 &  & Dummy & 20 & 0.127 & 0.167 \\
\cline{3-6}
 &  & Perfect & 2 & 0.507 & 0.616 \\
\cline{3-6}
 &  & \multirow[t]{3}{*}{Sparse} & 3 & 0.203 & 0.295 \\
 &  &  & 5 & 0.163 & 0.253 \\
 &  &  & 10 & 0.173 & 0.243 \\
\cline{1-6} \cline{2-6} \cline{3-6}
\multirow[t]{16}{*}{True} & \multirow[t]{8}{*}{CTE} & \multir